In [25]:
# this part of the codes fetches requirements
import hashlib
from phe import paillier
import random as rd
import numpy as np

In [26]:
listOfIDS = []
dictionaryOfPublicKeys = {}
dictionaryOfPrivateKeys = {}
dictionaryOfRandomNumbers = {}
dictionaryOfBids = {}
finalSubmittedDataFirstPhase = [] #will be sused to stored the final resuklt of the two bidder for bid submission
highestbidder = 0 
numberOfBidders = 0 
publicKeyAuc, privateKeyAuc = 0, 0
finalDataOfSecondPhase = []
productOfAllRandomNumbers = 1

In [27]:
#  this part of the code creates unique BID for all participants
def createBiddersAndKeys(n):
  global numberOfBidders,dictionaryOfPublicKeys,dictionaryOfPrivateKeys,publicKeyAuc,privateKeyAuc
  numberOfBidders = n
  for i in range(0,numberOfBidders):
    i = str(i)
    listOfIDS.append(hashlib.sha256(i.encode('utf-8')).hexdigest())
  i = 0
  publicKeyAuc, privateKeyAuc = paillier.generate_paillier_keypair(n_length= 64)
  for i in range(0, numberOfBidders):
    x,y = paillier.generate_paillier_keypair(n_length= 128)
    dictionaryOfPublicKeys[listOfIDS[i]],dictionaryOfPrivateKeys[listOfIDS[i]]  = x ,y
  i = 0

end of the Registration phase and start of Bid Submission Phase

In [28]:
# this part of the code generate random number and distinct bid values for all bidders and Auctioneer
def  createRandomNumberAndBids():
    global dictionaryOfBids,dictionaryOfRandomNumbers
    bids  = rd.sample(range(1, numberOfBidders+1 ), numberOfBidders)
    for i in range(0,numberOfBidders):
        r  = rd.randint(100,10000)
        dictionaryOfRandomNumbers[listOfIDS[i]] =  r
        dictionaryOfBids[listOfIDS[i]] = bids[i]
    i = 0
    print(dictionaryOfBids)
    print(dictionaryOfRandomNumbers)


In [29]:
# this PART OF THE CODE SIMULATES BIDDER GENERATING PRODUCT AND THAN SENDING TO NEXT BIDDER

def initFirstPhase():
    global finalSubmittedDataFirstPhase
    for i in range(0,numberOfBidders):
        id = listOfIDS[i]
        dataPerBidder= []
        encrytionOfBidAndRandomNumber = publicKeyAuc.raw_encrypt(dictionaryOfBids[id] * dictionaryOfRandomNumbers[id])
        # this part of code is forwarding the encrytionOfBidAndRandomNumber to all other bidders and after multiplication of the random number of the next
        # bidder it is sent to the Auctioneer immediately
        for j in range(0,numberOfBidders):
            if j != i:
                idOfNextBidder = listOfIDS[j]
                pubKeyNextBidder, priKeyNextBidder= dictionaryOfPublicKeys[idOfNextBidder],dictionaryOfPrivateKeys[idOfNextBidder]
                e = pubKeyNextBidder.raw_encrypt(encrytionOfBidAndRandomNumber)
                e = priKeyNextBidder.raw_decrypt(e)
                encrytionOfBidAndBothRandomNumber = e ** dictionaryOfRandomNumbers[idOfNextBidder]
                # this part of the code simulates the storing of the data of the bid summision to auctioneer data
                dataPerBidder.append(privateKeyAuc.raw_decrypt(encrytionOfBidAndBothRandomNumber))
            else:
                dataPerBidder.append(0)
        finalSubmittedDataFirstPhase.append(dataPerBidder)
    print(finalSubmittedDataFirstPhase)
    i,j = 0,0

implementation of the second phase here auctioneer reforward each packet to every bidder to multiply their random number.

In [30]:
# #this function works as the intermediate steps where we are requesting each bidder to multiplyt his random number to the given ciphertext
# def productOfRandomNumbers():
#     global productOfAllRandomNumbers
#     for k in range(numberOfBidders):
#         productOfAllRandomNumbers = productOfAllRandomNumbers * dictionaryOfRandomNumbers[listOfIDS[k]]


In [31]:
def everyBiddersRandomNumberMultiplication(encryptedAuctioneerRandomNumber):
    global productOfAllRandomNumbers
    print("////")
    output = encryptedAuctioneerRandomNumber
    for i in range(numberOfBidders):     
        print("zz")
        output  = pow(output,dictionaryOfBids[listOfIDS[i]])
        print(output)
    return output

In [32]:
def initSecondPhase():
    global finalDataOfSecondPhase
    auctioneerRandomNumber = rd.randint(2,2)
    print(auctioneerRandomNumber)
    for i in range(len(finalSubmittedDataFirstPhase)):
        dataPerBidder = []
        for j in range(numberOfBidders):
            if finalSubmittedDataFirstPhase[i][j] == 0 :
                dataPerBidder.append(0)
            else:
                encryptedValueByAuctioneer = publicKeyAuc.raw_encrypt(auctioneerRandomNumber)
                returnValue = everyBiddersRandomNumberMultiplication(encryptedValueByAuctioneer)
                dataPerBidder.append((privateKeyAuc.raw_decrypt(returnValue)))
        finalDataOfSecondPhase.append(dataPerBidder)
    i,j = 0,0


In [33]:
# def runSimulation():
#     print("//")
#     createBiddersAndKeys(2)
#     print("//")
#     createRandomNumberAndBids()
#     print("//")
#     initFirstPhase()
#     print("//")
#     initSecondPhase()
#     print("//")

In [34]:
createBiddersAndKeys(10)
createRandomNumberAndBids()



{'5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9': 10, '6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b': 7, 'd4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35': 8, '4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce': 2, '4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328cb08b5531fcacdabf8a': 4, 'ef2d127de37b942baad06145e54b0c619a1f22327b2ebbcfbec78f5564afe39d': 6, 'e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683': 3, '7902699be42c8a8e46fbbb4501726517e86b22c56a189f7625a6da49081b2451': 9, '2c624232cdd221771294dfbb310aca000a0df6ac8b66b696d90ef06fdefb64a3': 1, '19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7': 5}
{'5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9': 7733, '6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b': 8199, 'd4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35': 1031, '4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce

In [35]:
initFirstPhase()

[[0, 634028670, 79727230, 350150240, 279625280, 148318940, 538371460, 516873720, 418432630, 319682220], [443820069, 0, 59172183, 259875504, 207533088, 110079774, 399570066, 383614812, 310553523, 237262662], [63781784, 67625352, 0, 37346944, 29824768, 15819664, 57422576, 55129632, 44629928, 34097232], [70030048, 74250144, 9336736, 0, 32746496, 17369408, 63047872, 60530304, 49002016, 37437504], [7214257414722575426, 118590336, 14912384, 65492992, 0, 27741952, 13509088809459155, 96677376, 6325096132513338107, 13294548823094127531], [88991364, 94354092, 11864748, 52108224, 41612928, 0, 80118696, 76919472, 62269788, 47574072], [161511438, 171244314, 21533466, 94571808, 75523776, 40059348, 0, 139602024, 113014146, 86342724], [465186348, 493219044, 62020836, 272386368, 217524096, 115379208, 418806072, 0, 325504116, 248684904], [41843263, 44364789, 5578741, 24501008, 19566176, 10378298, 37671382, 36167124, 0, 22369074], [159841110, 169473330, 21310770, 93593760, 74742720, 39645060, 143904540, 

In [36]:
initSecondPhase()

In [ ]:
random = publicKeyAuc.raw_encrypt(2)
output = 1
for i in range(10):
    output  = pow(output,dictionaryOfBids[listOfIDS[i]])
print(privateKeyAuc.raw_decrypt(output))   

In [ ]:

for i in range(len(finalDataOfSecondPhase)):
    print(finalDataOfSecondPhase[i])
i = 0

In [ ]:
for i in range(len(finalSubmittedDataFirstPhase)):
    print(finalSubmittedDataFirstPhase[i])
i = 0

[0, 50947920, 77521920, 94850640, 158751840, 126961920, 179096400, 208488480, 231651120, 129903600]
[25473960, 0, 32316480, 39540285, 66178710, 52926480, 74659725, 86912370, 96568155, 54152775]
[46513152, 38779776, 0, 72196992, 120836352, 96638976, 136321920, 158694144, 176324736, 98878080]
[9485064, 7908057, 12032832, 0, 24641214, 19706832, 27799065, 32361258, 35956527, 20163435]
[47625552, 39707226, 60418176, 73923642, 0, 98950176, 139582170, 162489444, 180541686, 101242830]
[50784768, 42341184, 64425984, 78827328, 131933568, 0, 148841280, 173268096, 192517824, 107958720]
[143277120, 119455560, 181762560, 222392520, 372219120, 297682560, 0, 488834640, 543143160, 304579800]
[145941936, 121677318, 185143168, 226528806, 379142036, 303219168, 427730310, 0, 553245098, 310244690]
[46330224, 38627262, 58774912, 71913054, 120361124, 96258912, 135785790, 158070028, 0, 98489210]
[116913240, 97474995, 148317120, 181470915, 303728490, 242907120, 342652275, 398886030, 443201445, 0]
